In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython.display as ipd

wav_path = "lotek_owl_data/test.wav"
ipd.Audio(wav_path)

In [ ]:
from data_utils import display_audio_file


display_audio_file(wav_path)

In [ ]:
from data_utils import display_melspec, display_zero_crossings


display_zero_crossings()

In [ ]:
from data import load_data


batch_sz = 512
debug = False
spec_height = 750
owlet_train, owlet_test, owlet_dataset = load_data(
    "lotek_owl_data",
    train_test_split=[1.0, 0],
    batch_sz=batch_sz,
    debug=debug,
    spec_height=spec_height
)

---
# How does it work?

<img src="./img/explain_model.png" alt="diagram" width="1200">

---

<img src="./img/explain_loss.png" alt="loss" width="600">

---

# This is where the training code would go, but in the interest of time ...
---

In [ ]:
import torch


# display trained embeds
def create_embeds(model, dataloader):
    embeds = []
    specs = []
    specs_og = []
    for batch in dataloader:
        data_specs, og_specs = batch
        specs_og += og_specs.unbind()
        specs += data_specs.unbind()
        data_specs = data_specs.cuda()
        embeds_batch = model(data_specs.cuda())
        embeds.append(embeds_batch.detach().cpu())
    embeds = torch.cat(embeds)
    return embeds, specs, specs_og

In [ ]:
from model import OwlNet


drop = 0.0
embed_sz = 64
spec_height = 750

model_name = "model_3584.datapoints_105.epochs.pth"
owlnet_dict = torch.load(model_name)
owlnet = OwlNet(embed_sz, drop).cuda()
owlnet.load_state_dict(owlnet_dict)
owlnet.eval()
pass


In [ ]:
from utils import VisualiserInteractive
from data import CollateFunc


collate_func = CollateFunc(spec_height=spec_height)
embeddings, melspecs, melspecs_og = create_embeds(owlnet, owlet_train)
vis = VisualiserInteractive(embeddings, melspecs, melspecs_og)
vis.show()